In [268]:
import re
        
def _read(fname):
    d = {}
    with open(fname,'r') as f:
        for l in f:
            ll = l.strip()
            if ll:
                if ll[:4] == "Tile":
                    no = int(re.findall("\d+", ll)[0])
                    d[no] = []
                else:
                    d[no].append(ll.replace("#","1").replace(".","0"))
    return d
    
t = _read("t.txt")
i = _read("i.txt")

In [269]:
def flip(tile, updown=True):
    t = []
    if updown:
        for i in tile[::-1]:
            t.append(i)
    else:
        for i in tile:
            t.append(i[::-1])
    return t

def rotate(tile, left=True):
    t = tile[::]
    l = len(tile)-1
    if left:
        for line in range(l+1):
            t[line] = ""
            for col in range(l+1):
                t[line] += tile[col][l-line]
    else:
        for line in range(l+1):
            t[line] = ""
            for col in range(l+1):
                t[line] += tile[l-col][line]
        
    return t

def encode(tile):
    up = int(tile[0], base=2)
    left = int("".join([l[0] for l in tile]), base=2)
    right = int("".join([l[9] for l in tile]), base=2)
    button = int(tile[9], base=2)
    return up, right, button, left

def all_variants(tile):
    a=[]
    for i in range(2):
        for i in range(4): 
            tile = rotate(tile)
            a.append(tile)
        tile = flip(tile)
    return a

def encode_all(tile):
    a=[]
    for i in range(2):
        for i in range(4): 
            tile = rotate(tile)
            a.append(encode(tile))
        tile = flip(tile)
    return a

# t1 links t2 rechts
def match_left(t1, t2):
    return t1[1] == t2[3]

# t1 rechts t2 links
def match_right(t1, t2):
    return t1[3] == t2[1]

# t1 oben t2 unten
def match_up(t1, t2):
    return t1[2] == t2[0]

def match_buttom(t1, t2):
    return t1[0] == t2[2]



In [270]:
def translate(tiles):
    r = {}
    v = {}
    for no, tile in tiles.items():
        r[no] = encode_all(tile)
        v[no] = all_variants(tile)
    return r, v

def find(pic, pool, used, pos1, pos2, dim):
    up = pos1-1 
    left = pos2-1
    #print("{} {}".format(pos1,pos2))
    for no, tile in pool.items():
        if no in used:
            continue
        for version in tile:
            if up > -1:
                if not match_up(pic[(up, pos2)], version):
                    continue
            if left > -1:
                if not match_left(pic[(pos1, left)], version):
                    continue
            pic[(pos1, pos2)] = version
            used[no]=(pos1, pos2)
            if pos2 < dim-1:
                if find(pic, pool, used, pos1, pos2+1, dim):
                    return True
            elif pos1 < dim-1:
                if find(pic, pool, used, pos1+1, 0, dim):
                    return True
            else:
                return True
            del used[no]
    return False

tt,tv = translate(t)  
ii,iv = translate(i)  

In [271]:
def cut(t):
    return [tt[1:-1] for tt in t[1:-1]]

def get_tile(i,j, used, pics, variants, codes):
    for no, pos in used.items():
        if pos == (i,j):
            break
    v = pics[pos]
    for vno, var in enumerate(codes[no]):
        #print("{} {}".format(v,var))
        if v == var:
            break
    else:
        print("no found")
    
    return cut(variants[no][vno])

def glue(a, b):
    r = []
    for i in range(len(a)):
        r.append(a[i]+b[i])
    return r
    
def part1(_input):
    dim = int(len(_input)**0.5)
    
    codes, variants = translate(_input)
    # pool list of tiles
    used = {}
    pics={}
    find(pics, codes, used, 0, 0, dim)
    prod=1

    for no, pos in  used.items():
            if pos[0] in [0, dim-1] and pos[1] in [0, dim-1]:
                prod *=int(no)
    
    image = []
    for i in range(dim):
        row = get_tile(i,0,used, pics, variants, codes)
        for j in range(1,dim):
            row = glue(row, get_tile(i, j,used, pics, variants, codes))
        image += row
        
    return prod, image

In [272]:
ip, iimage = part1(i)
tp, timage = part1(t)

In [273]:
tp, ip 

(20899048083289, 140656720229539)

In [285]:
def replace(s, position, character):
    return s[:position] + character + s[position+1:]

def monster():
    # dim = 3x20
    #  01234567890123456789
    a="00000000000000000010"
    b="10000110000110000111"
    c="01001001001001001000"
    return int(a+b+c, base=2)

def search(image, i, j, monster):
    test = int(image[i][j:j+20] + image[i+1][j:j+20] + image[i+2][j:j+20], base=2)
    #print("{}  {}  {} ".format(test, monster, test & monster))
    return monster == (test & monster)

def mark(image, i, j):
    image[i] = replace(image[i], j+18, "a")
    image[i+1] = replace(image[i+1], j+0, "a")
    image[i+1] = replace(image[i+1], j+5, "a")
    image[i+1] = replace(image[i+1], j+6, "a")
    image[i+1] = replace(image[i+1], j+11, "a")
    image[i+1] = replace(image[i+1], j+12, "a")
    image[i+1] = replace(image[i+1], j+17, "a")
    image[i+1] = replace(image[i+1], j+18, "a")
    image[i+1] = replace(image[i+1], j+19, "a")
    image[i+2] = replace(image[i+2], j+1, "a")
    image[i+2] = replace(image[i+2], j+4, "a")
    image[i+2] = replace(image[i+2], j+7, "a")
    image[i+2] = replace(image[i+2], j+10, "a")
    image[i+2] = replace(image[i+2], j+13, "a")
    image[i+2] = replace(image[i+2], j+16, "a")
    return image
    
                       
def find_monster(image):
    m = monster()
    
    cimage = image.copy()
    rows=len(image)
    cols=len(image[0])
    for i in range(rows-2):
        for j in range(cols-19):
            if search(image, i,j, m):
                cimage = mark(cimage, i, j)
    #print(cimage)
    w = 0
    for i in range(rows):
        for j in range(cols):
            if cimage[i][j] == "1":
                       w += 1
    return w

def count_water(image): 
    l = []
    test_image = image
    for ii in range(2):
        for jj in range(4): 
            test_image = rotate(test_image)
            l.append(find_monster(test_image))    
        test_image = flip(test_image)
    return min(l)


In [286]:
count_water(timage)

273

In [287]:
count_water(iimage)

1885